In [ ]:
import sys, os
sys.path.append(os.path.abspath("../src"))



from llm.gemini_utils import load_gemini_model
from llm.prompt_templates import build_prompt

from dotenv import load_dotenv
load_dotenv()

model = load_gemini_model()

def clean_sql(text):
    return text.replace("```sql", "").replace("```", "").strip()


# Ask your question
question = "Which player scored the most runs in 2016 IPL?"
prompt = build_prompt(question)

# Gemini generates SQL
response = model.generate_content(prompt)
sql = response.text.strip()

if sql.startswith("```sql"):
    sql = sql.strip("```sql").strip("```").strip()
elif sql.startswith("```"):
    sql = sql.strip("```").strip()
    
print("🔍 Gemini-generated SQL:\n", sql)


🔍 Gemini-generated SQL:
 SELECT batsman, SUM(batsman_runs) AS total_runs
FROM deliveries
JOIN matches ON deliveries.match_id = matches.id
WHERE season = 2016
GROUP BY batsman
ORDER BY total_runs DESC
LIMIT 1;


In [4]:
import pymysql
import pandas as pd

def run_query(sql):
    conn = pymysql.connect(
        host="localhost",
        user="root",
        password=os.getenv("MYSQL_PASSWORD"),
        database="cricstats"
    )
    df = pd.read_sql(sql, conn)
    conn.close()
    return df
result_df = run_query(sql)
result_df


c:\Users\singh_7l8q9pm\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
C:\Users\singh_7l8q9pm\AppData\Local\Temp\ipykernel_20072\4124251789.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


,batsman,total_runs
0,V Kohli,973.0


In [ ]:
questions = [
  "How many matches did KKR win in 2016?",
  "What is the average score in matches at Wankhede?",
  "List top 3 players dismissed most by Bhuvneshwar Kumar"
]

for q in questions:
    prompt = build_prompt(q)
    sql = sql = clean_sql(model.generate_content(prompt).text)

    print(f" NLQ: {q}\n SQL: {sql}\n")
    try:
        result = run_query(sql)
        display(result.head())
    except Exception as e:
        print(f" Error: {e}")


🧠 NLQ: How many matches did KKR win in 2016?
📄 SQL: SELECT COUNT(*)
FROM matches
WHERE season = 2016 AND winner = 'Kolkata Knight Riders';



C:\Users\singh_7l8q9pm\AppData\Local\Temp\ipykernel_15436\4124251789.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


,COUNT(*)
0,8


🧠 NLQ: What is the average score in matches at Wankhede?
📄 SQL: SELECT AVG(total_runs)
FROM deliveries AS d
JOIN matches AS m ON d.match_id = m.id
WHERE m.venue = 'Wankhede Stadium';



C:\Users\singh_7l8q9pm\AppData\Local\Temp\ipykernel_15436\4124251789.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


,AVG(total_runs)
0,1.3149


🧠 NLQ: List top 3 players dismissed most by Bhuvneshwar Kumar
📄 SQL: SELECT player_dismissed
FROM deliveries
WHERE bowler = 'Bhuvneshwar Kumar' AND player_dismissed IS NOT NULL
GROUP BY player_dismissed
ORDER BY COUNT(*) DESC
LIMIT 3;



C:\Users\singh_7l8q9pm\AppData\Local\Temp\ipykernel_15436\4124251789.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


,player_dismissed


In [5]:
import sys, os
sys.path.append(os.path.abspath("../src"))

from llm.prompt_templates import render_template

print(render_template("top_scorers", season=2016, limit=5))
print(render_template("team_wins", team="RCB", season=2014))


SELECT batsman, SUM(batsman_runs) AS total_runs
        FROM deliveries
        JOIN matches ON deliveries.match_id = matches.id
        WHERE season = 2016
        GROUP BY batsman
        ORDER BY total_runs DESC
        LIMIT 5;
SELECT COUNT(*) AS wins
        FROM matches
        WHERE winner = 'RCB' AND season = 2014;


In [6]:
run_query(render_template("top_scorers", season=2016, limit=5))

C:\Users\singh_7l8q9pm\AppData\Local\Temp\ipykernel_20072\4124251789.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


,batsman,total_runs
0,V Kohli,973.0
1,DA Warner,848.0
2,AB de Villiers,687.0
3,S Dhawan,501.0
4,G Gambhir,501.0


In [3]:
import sys, os
sys.path.append(os.path.abspath("../src"))
from agents.multitool_agent import get_multitool_agent

In [4]:
# run Compare V Kholi vs S Dhawan
agent = get_multitool_agent()
question = "Compare V Kholi vs S Dhawan"
response = agent.run(question)

d:\BITS Pilani\Year 4\Sem 4-2\LLM\Project\cricket-stats-gpt\src\agents\multitool_agent.py:31: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(
C:\Users\singh_7l8q9pm\AppData\Local\Temp\ipykernel_2952\2093600427.py:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.ru



> Entering new AgentExecutor chain...
I need to compare two players.
Action: get_comparison
Action Input: V Kohli vs S Dhawan

d:\BITS Pilani\Year 4\Sem 4-2\LLM\Project\cricket-stats-gpt\src\analytics\comparisons.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, connect_db())



Observation: Final Answer:
  player  innings  total_runs  fours  sixes  balls_faced  strike_rate
S Dhawan     3005      3561.0  401.0   71.0         3005       118.50
 V Kohli     3494      4423.0  384.0  160.0         3494       126.59
Thought:I have the comparison data.
Final Answer:
```json
[
  {
    "player": "S Dhawan",
    "innings": 3005,
    "total_runs": 3561.0,
    "fours": 401.0,
    "sixes": 71.0,
    "balls_faced": 3005,
    "strike_rate": 118.50
  },
  {
    "player": "V Kohli",
    "innings": 3494,
    "total_runs": 4423.0,
    "fours": 384.0,
    "sixes": 160.0,
    "balls_faced": 3494,
    "strike_rate": 126.59
  }
]
```

> Finished chain.


In [5]:
# run Compare V Kholi vs S Dhawan
agent = get_multitool_agent()
question = "Wins by Mumbai Indians in 2016 IPL and the teams they beat" 
response = agent.run(question)



> Entering new AgentExecutor chain...
I need to find the wins by Mumbai Indians in 2016 and the teams they beat. I can use the `get_team_wins` tool to get the number of wins per season for Mumbai Indians. Since I also need to know the teams they beat, I will need to use SQL fallback.
Action: get_team_wins
Action Input: Mumbai Indians
Observation: Final Answer:
 season  wins
   2008     7
   2009     5
   2010    11
   2011    10
   2012    10
   2013    13
   2014     7
   2015    10
   2016     7
   2017    12
Thought:

d:\BITS Pilani\Year 4\Sem 4-2\LLM\Project\cricket-stats-gpt\src\analytics\team.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connect_db())


I have the number of wins for Mumbai Indians in 2016. Now I need to find the teams they beat in 2016. I will use SQL fallback for this.
Action: sql_fallback
Action Input: SELECT match_id, team1, team2, winner FROM matches WHERE (team1 = 'Mumbai Indians' OR team2 = 'Mumbai Indians') AND season = 2016

> Entering new SQL Agent Executor chain...
```sql
SELECT
  matches.id,
  matches.team1,
  matches.team2,
  matches.winner
FROM matches
WHERE
  (    matches.team1 = 'Mumbai Indians'
    OR matches.team2 = 'Mumbai Indians'
  )
  AND matches.season = 2016
LIMIT 100
```

> Finished chain.

Observation: Final Answer:
SQL used:
SELECT matches.id, matches.team1, matches.team2, matches.winner FROM matches WHERE ( matches.team1 = 'Mumbai Indians' OR matches.team2 = 'Mumbai Indians' ) AND matches.season = 2016 LIMIT 100

Results:
 id                       team1                   team2                  winner
577              Mumbai Indians Rising Pune Supergiants Rising Pune Supergiants
581       Ko

d:\BITS Pilani\Year 4\Sem 4-2\LLM\Project\cricket-stats-gpt\src\agents\langchain_sql_agent.py:64: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_cleaned, connect_db())


I have the number of wins for Mumbai Indians in 2016 (7) and the teams they beat in 2016.
Final Answer: Mumbai Indians won 7 matches in 2016. They beat Kolkata Knight Riders twice, Royal Challengers Bangalore twice, Kings XI Punjab, and Delhi Daredevils. They lost to Rising Pune Supergiants, Gujarat Lions, and Sunrisers Hyderabad.

> Finished chain.


In [ ]:
import pandas as pd
import time
from agents.multitool_agent import get_multitool_agent

# Initialize LangChain agent
agent = get_multitool_agent()

# Helper to parse agent response
def parse_response(response_str):
    tool_name = None
    sql_query = None
    final_result = None
    lines = response_str.splitlines()

    for i, line in enumerate(lines):
        if "Action:" in line:
            tool_name = line.split("Action:")[1].strip()
        if "```sql" in line:
            sql_lines = []
            for j in range(i + 1, len(lines)):
                if lines[j].startswith("```"):
                    break
                sql_lines.append(lines[j])
            sql_query = "\n".join(sql_lines)
        if "Final Answer:" in line:
            final_result = "\n".join(lines[i+1:]).strip()
            break

    return sql_query or tool_name, final_result

# === INPUT: Replace this with the desired batch ===
nlp_batch = [
     "Compare V Kohli vs RG Sharma",
    "Compare MS Dhoni vs S Dhawan",
    "Compare AB de Villiers vs CH Gayle",
    "List top 5 run scorers of 2016",
    "Top 3 run scorers in 2015",
]

# Output results
results = []

for question in nlp_batch:
    try:
        response = agent.run(question)
        tool_or_sql, final = parse_response(response)
        results.append({
            "nlp": question,
            "sql_result_llm": tool_or_sql,
            "llm_result": final
        })
    except Exception as e:
        results.append({
            "nlp": question,
            "sql_result_llm": "ERROR",
            "llm_result": str(e)
        })
    time.sleep(1)

# Convert to DataFrame and optionally save
df = pd.DataFrame(results)
print(df)

# Optional: save to CSV
# df.to_csv("batch_results.csv", index=False)


d:\BITS Pilani\Year 4\Sem 4-2\LLM\Project\cricket-stats-gpt\src\agents\multitool_agent.py:31: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(
C:\Users\singh_7l8q9pm\AppData\Local\Temp\ipykernel_20712\465415081.py:45: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.r



> Entering new AgentExecutor chain...
I need to compare two players, so I should use the get_comparison tool.
Action: get_comparison
Action Input: V Kohli vs S Dhawan

d:\BITS Pilani\Year 4\Sem 4-2\LLM\Project\cricket-stats-gpt\src\analytics\comparisons.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, connect_db())



Observation: Final Answer:
  player  innings  total_runs  fours  sixes  balls_faced  strike_rate
S Dhawan     3005      3561.0  401.0   71.0         3005       118.50
 V Kohli     3494      4423.0  384.0  160.0         3494       126.59
Thought:I now have the comparison data.
Final Answer:
```
player  innings  total_runs  fours  sixes  balls_faced  strike_rate
S Dhawan     3005      3561.0  401.0   71.0         3005       118.50
 V Kohli     3494      4423.0  384.0  160.0         3494       126.59
```

> Finished chain.


> Entering new AgentExecutor chain...
I need to find the average runs per match for Rohit Sharma. I don't have a direct tool for that. I can try to get a comparison with another player to see if I can get some stats. If that doesn't work, I'll need to use SQL.
Action: get_comparison
Action Input: Rohit Sharma vs Virat Kohli

d:\BITS Pilani\Year 4\Sem 4-2\LLM\Project\cricket-stats-gpt\src\analytics\comparisons.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, connect_db())


...

d:\BITS Pilani\Year 4\Sem 4-2\LLM\Project\cricket-stats-gpt\src\agents\langchain_sql_agent.py:64: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_cleaned, connect_db())


Thought: The previous query failed. I need to try a different approach. I can't directly get economy rates using the provided tools. I will try to find top wicket takers and then use SQL to get their economy.
Action: get_top_wicket_takers
Action Input: 2023

d:\BITS Pilani\Year 4\Sem 4-2\LLM\Project\cricket-stats-gpt\src\analytics\metrics.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connect_db())



Observation: Final Answer:
Empty DataFrame
Columns: [bowler, wickets]
Index: []
Thought:I am unable to answer this question with the available tools. The tools do not have the functionality to directly find the top 5 bowlers by economy. The SQL fallback also failed.
Final Answer: I am sorry, I cannot answer this question with the available tools.

> Finished chain.


KeyboardInterrupt: 

In [1]:
import pandas as pd

# Load the file
df = pd.read_csv("../data/try4_results.csv")  # Adjust path if needed

# Extract the NLP prompts
nlp_list = df["nlp"].dropna().tolist()

# Split into batches of 5
nlp_batches = [nlp_list[i:i+5] for i in range(0, len(nlp_list), 5)]

# Generate Python array initializations
for i, batch in enumerate(nlp_batches):
    print(f"# Batch {i+1}")
    print("nlp_batch = [")
    for prompt in batch:
        print(f'    "{prompt}",')
    print("]\n")


c:\Users\singh_7l8q9pm\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# Batch 1
nlp_batch = [
    "Compare V Kohli vs RG Sharma",
    "Compare MS Dhoni vs S Dhawan",
    "Compare AB de Villiers vs CH Gayle",
    "List top 5 run scorers of 2016",
    "Top 3 run scorers in 2015",
]

# Batch 2
nlp_batch = [
    "Show top 10 scorers from 2017 season",
    "Top 5 wicket takers of 2014",
    "Top 5 wicket takers of 2011",
    "Leading wicket taker in 2013",
    "Average score at Wankhede Stadium",
]

# Batch 3
nlp_batch = [
    "Average score at Eden Gardens",
    "Average score at M Chinnaswamy Stadium",
    "Wins per season for Mumbai Indians",
    "Wins per season for Chennai Super Kings",
    "Powerplay runs for Royal Challengers Bangalore in 2016",
]

# Batch 4
nlp_batch = [
    "Powerplay totals for Delhi Daredevils in 2015",
    "Win percentage of Mumbai Indians",
    "Win percentage of Chennai Super Kings",
    "Match summary for match 1",
    "Match summary for match 2",
]

# Batch 5
nlp_batch = [
    "Who has the highest strike rate in 2016?",
    "B